# Lab 1: 前提条件とインフラストラクチャのセットアップ

## 概要
ワークショップのすべての前提条件を確認し、AWS 上に CRM アプリケーションスタック（EC2 + NGINX + DynamoDB）をデプロイします。

## 目標
**パート 1: 前提条件**
- Python バージョンの確認（3.10 以上）
- AWS アカウントと認証情報の確認
- ワークショップの依存関係をインストール
- Bedrock AgentCore SDK とスターターツールキットの確認
- Bedrock モデルアクセスのテスト
- 共有コンテキスト用の Agent Memory のセットアップ
- Amazon Cognito を使用したユーザーとエージェント ID のセットアップ

**パート 2: インフラストラクチャのセットアップ**
- AWS インフラストラクチャのプロビジョニング: EC2 インスタンス、NGINX、DynamoDB、CloudWatch
- サンプル CRM アプリケーションのデプロイ
- 障害をシミュレートするための障害注入スクリプトの作成
- CloudWatch モニタリングのセットアップ
- インフラストラクチャが稼働しアクセス可能であることを確認

## 学習内容
- ワークショップの前提条件とセットアップワークフロー
- インシデント対応テストのための障害注入パターン
- 診断用の CloudWatch ログとメトリクスのセットアップ

## 1. Python バージョンの確認

In [ ]:
import sys
print(f"Pythonバージョン: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
assert sys.version_info >= (3, 10), "Python 3.10以上が必要です"
print("✅ Pythonバージョンチェックに合格しました")

## 2. ワークショップの依存関係をインストール

In [ ]:
%pip install -q -r requirements.txt
print("✅ ワークショップの依存関係がインストールされました")

## 3. AWS 設定の確認

In [ ]:
import boto3
from lab_helpers.config import AWS_REGION, AWS_PROFILE, MODEL_ID, WORKSHOP_NAME
from lab_helpers.lab_01.infrastructure import get_app_url

# 設定を表示
print(f"ワークショップ名: {WORKSHOP_NAME}")
print(f"AWSリージョン: {AWS_REGION}")
print(f"モデルID: {MODEL_ID}\n")

# AWS 認証情報を確認
session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION)
sts = session.client('sts')
identity = sts.get_caller_identity()

print(f"✅ AWSアカウント: {identity['Account']}")
print(f"✅ AWSユーザー/ロール: {identity['Arn']}")

## 4. Bedrock モデルアクセスのテスト

In [ ]:
import boto3
from lab_helpers.config import AWS_REGION, MODEL_ID, AWS_PROFILE

session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION)
bedrock = session.client('bedrock', region_name=AWS_REGION)

# モデルアクセスを確認
try:
    model = bedrock.get_foundation_model(modelIdentifier=MODEL_ID)
    print(f"モデルID: {MODEL_ID}")
    print(f"✅ Bedrockモデルアクセスを確認しました")
except Exception as e:
    print(f"❌ モデルアクセスエラー: {e}")
    raise

## 5. AgentCore コンポーネントの確認

In [ ]:
import importlib

packages = ['bedrock_agentcore', 'strands', 'boto3', 'pydantic']

for package in packages:
    try:
        mod = importlib.import_module(package)
        version = getattr(mod, '__version__', 'installed')
        print(f"✅ {package:<20} {version}")
    except ImportError:
        print(f"❌ {package:<20} 見つかりません")

print("\n✅ すべてのコアパッケージが確認されました")

## まとめ
✅ すべての前提条件が確認されました。Lab 1: インフラストラクチャのセットアップと障害注入に進む準備ができました。

## パート 1.5: Cognito のセットアップ（Lab 3-5 の認証）

### 概要

このセクションでは、Lab 3-5 で使用する認証インフラストラクチャとして AWS Cognito をセットアップします:

**作成するもの:**
- Cognito User Pool: `aiml301-UserPool`
- **2つのユーザーグループ**（新規）:
  - **developers**: 修復プランを作成するユーザー
  - **approvers**: プランを承認・実行するユーザー
- **2つのアプリクライアント**:
  - **User Auth Client**（パブリック）: OAuth サポート付きのエンドユーザー認証用
  - **M2M Client**（機密）: Gateway から Runtime へのサービス間認証用
- **Resource Server**: きめ細かい認可のためのカスタムスコープ（`mcp.invoke`、`runtime.access`）
- **User Pool Domain**: OAuth2 トークンエンドポイント
- **2つのテストユーザー**:
  - **Developer User**: `testuser@aiml301.example.com`（`developers` グループのメンバー）
  - **Approver User**: `approver@aiml301.example.com`（`approvers` グループのメンバー）

**認証フロー:**
1. **User Auth**（Client → Gateway）: エンドユーザーが認証情報で認証し、グループメンバーシップを含む JWT トークンを受け取る
2. **M2M Auth**（Gateway → Runtime）: Gateway がクライアント認証情報グラントを使用して Runtime アクセス用の M2M トークンを取得

**マルチアクターワークフロー（Lab-03）:**
- **Developer** がログイン → 修復プランを作成 → ブロックされる（承認が必要）
- **Approver** がログイン → 保留中のインシデントを発見 → プランをレビュー → 実行を承認
- **Developer** が戻る → 共有メモリで承認を確認 → 承認されたステップを実行

**JWT トークンクレーム:**
セットアップ後、JWT ID トークンには以下が含まれます:
```json
{
  "email": "developer@aiml301.example.com",
  "cognito:username": "developer@aiml301.example.com",
  "cognito:groups": ["developers"],
  "sub": "uuid",
  "scope": "openid profile email custom-scopes"
}
```

**グループを使用する理由**
- **ロールベースの認可**: Gateway は実行を許可する前にユーザーが `approvers` グループに所属しているか確認できる
- **インシデントルーティング**: 保留中のインシデントについて承認者のみに通知
- **監査証跡**: Memory レコードにどのロールが各アクションを実行したかを表示
- **アクター識別**: `email` クレームが UUID の代わりに読みやすい actor_id を提供

### 目標

✅ 一元化された認証のための Cognito インフラストラクチャのセットアップ
✅ ロールベースのアクセス制御のためのユーザーグループの作成
✅ ユーザーベースとサービス間のデュアル認証モードの有効化
✅ きめ細かい認可スコープの作成
✅ リッチな JWT ID トークンのための OAuth フローの有効化
✅ 後続の Lab で使用するための SSM Parameter Store への設定の保存

#### 1. Cognito セットアップの実行

In [ ]:
from lab_helpers.cognito_setup import setup_cognito_complete

# Cognito セットアップワークフローを完全に実行
cognito_config = setup_cognito_complete()

print("\n" + "="*70)
print("COGNITOセットアップ完了")
print("="*70)
print("CognitoユーザープールID: ", cognito_config['user_pool_id'])

## パート 1.6: Lab 2-5 用の Memory セットアップ

このセクションでは、すべてのエージェント Lab（2-5）で会話履歴とセッション管理に使用される共有 AgentCore Memory リソースを作成します。

### 作成するもの:
- 7日間の有効期限を持つ AgentCore Memory リソース
- Lab 2-5 から簡単にアクセスできるよう Parameter Store に memory_id を保存
- Lab 2-4 での静的セッション追跡用のデフォルトセッション ID を保存

### 重要な学習ポイント:
Memory により、エージェント呼び出し間およびマルチターン会話でのコンテキストの永続化が可能になります。すべての Lab でこの単一の Memory リソースを共有します。

### 目標
✅ AgentCore Memory リソースの作成
✅ Parameter Store への Memory 設定の保存
✅ 下流のエージェントの会話履歴読み込みの有効化

In [ ]:
### 1.6.1: AgentCore Memory リソースを作成

from bedrock_agentcore.memory import MemoryClient
from lab_helpers.constants import PARAMETER_PATHS
from datetime import datetime

memory_client = MemoryClient(region_name=AWS_REGION)
memory_name = f"{PARAMETER_PATHS['memory']['memory_name_prefix']}_{datetime.now().strftime('%Y%m%d%H%M%S')}"

print(f"メモリを作成中: {memory_name}")
memory = memory_client.create_memory_and_wait(
    name=memory_name,
    description="SRE Agent Shared Short-Term Memory for Labs 2-5",
    strategies=[],
    event_expiry_days=7,
    max_wait=600,
    poll_interval=10
)

memory_id = memory['id']
print(f"✅ メモリを作成しました: {memory_id} (ステータス: ACTIVE, 有効期限: 7日間)")

In [ ]:
### 1.6.2: Memory 設定を Parameter Store に保存

from lab_helpers.parameter_store import put_parameter

# Lab 2-5 用に memory_id を保存
put_parameter(
    PARAMETER_PATHS['memory']['memory_id'],
    memory_id,
    description="Memory ID for agent conversation history",
    region_name=AWS_REGION
)

# Lab 2-4 用にデフォルトセッション ID を保存
put_parameter(
    PARAMETER_PATHS['memory']['default_session_id'],
    "crm-session-id",
    description="Default session ID for Labs 2-4",
    region_name=AWS_REGION
)

print(f"✅ PSMキーが保存されました:")
print(f"   • {PARAMETER_PATHS['memory']['memory_id']} = {memory_id}")
print(f"   • {PARAMETER_PATHS['memory']['default_session_id']} = crm-session-id")

### まとめ: Memory セットアップ完了

✅ **AgentCore Memory リソースが作成されました**
- すべての Lab（2-5）で共有される単一の Memory リソース
- コスト管理のための自動 7 日間有効期限
- マルチターン会話とコンテキスト読み込みをサポート

✅ **Parameter Store の設定**
- Lab 2-5 で取得するための Memory ID を保存
- Lab 2-4 で利用可能なデフォルトセッション ID
- 一元設定パターンに準拠

**次のステップ:**
- Lab 2: memory_id を取得し、Memory フックを初期化
- Lab 3-4: 修復/予防エージェントに同じ Memory を使用
- Lab 5: 共有 Memory を使用したマルチエージェントオーケストレーション

## パート 2: インフラストラクチャのセットアップと CRM アプリケーションのデプロイ

インフラストラクチャのセットアップと CRM アプリケーションのデプロイは、ワークショップセットアップの一部として自動化されています。

次のセクションに進んでください。

In [ ]:
# ポート 80 と 8080 の両方で URL を試す
print(f"CRMアプリUIにアクセスするにはこちらをクリック: '{get_app_url()}'")

## 1. 障害注入ユーティリティのセットアップ

このセクションでは、インフラストラクチャ障害を注入するツールを準備し、デプロイメントに既に組み込まれている事前設定された障害をレビューします。このワークショップには、包括的な SRE トレーニング用に **合計4つの障害** が含まれています:

- **障害 1: DynamoDB スロットリング** - テーブル容量を削減して ProvisionedThroughputExceededException をトリガー
- **障害 2: IAM 権限の問題** - EC2 ロールの権限を制限して AccessDenied エラーを発生させる

これらの障害は、ワークショップ全体を通じて、異なる障害モードと検出方法に対する SRE エージェントの診断機能をテストするために使用されます。

In [ ]:
from lab_helpers.lab_01.fault_injection import (
    initialize_fault_injection,
    inject_dynamodb_throttling,
    inject_iam_permissions,
)

# AWS クライアントを初期化し、SSM からインフラストラクチャリソース ID を取得
print("障害注入ユーティリティを初期化中...")
resources = initialize_fault_injection(AWS_REGION, AWS_PROFILE)

print(f"\n検出されたインフラストラクチャリソース:")
print(f"  Nginxインスタンス: {resources.get('nginx_instance_id', '見つかりません')}")
print(f"  アプリインスタンス: {resources.get('app_instance_id', '見つかりません')}")
print(f"  CRM Activitiesテーブル: {resources.get('crm_activities_table_name', '見つかりません')}")
print(f"  CRM Customersテーブル: {resources.get('crm_customers_table_name', '見つかりません')}")
print(f"  CRM Dealsテーブル: {resources.get('crm_deals_table_name', '見つかりません')}")
print(f"  EC2ロール: {resources.get('ec2_role_name', '見つかりません')}")
print(f"  パブリックALB DNS: {resources.get('public_alb_dns', '見つかりません')}")

print("\n✅ 障害注入ユーティリティの準備ができました")

## 2. インフラストラクチャの確認

障害を注入する前に、CloudFormation スタックが必要なすべてのリソースを作成し、それらが正常であることを確認しましょう。

In [ ]:
from lab_helpers.lab_01.infrastructure import (
    verify_ec2_instances,
    verify_dynamodb_tables,
    verify_alb_health,
    verify_cloudwatch_logs
)

print("インフラストラクチャコンポーネントを確認中...\n")

# EC2 インスタンスが実行中か確認
ec2_status = verify_ec2_instances(resources, AWS_REGION, AWS_PROFILE)

# DynamoDB テーブルが存在しアクセス可能か確認
dynamodb_status = verify_dynamodb_tables(resources, AWS_REGION, AWS_PROFILE)

# ALB ターゲットが正常か確認
alb_status = verify_alb_health(resources, AWS_REGION, AWS_PROFILE)

# CloudWatch ロググループが存在するか確認
logs_status = verify_cloudwatch_logs(AWS_REGION, AWS_PROFILE)

if all([ec2_status, dynamodb_status, alb_status, logs_status]):
    print("\n✅ すべてのインフラストラクチャコンポーネントが確認され、正常です")
else:
    print("\n⚠️  一部のインフラストラクチャコンポーネントの確認に失敗しました")

## 3. 障害注入のテストと事前設定された障害のレビュー

このセクションでは、2つのインフラストラクチャ障害を注入し、デプロイメントに事前設定されている追加の2つの障害をレビューします。これらの **4つの障害** を合わせて、診断エージェントのための包括的なトレーニングシナリオを提供します。

### 障害 1: DynamoDB スロットリング

**概要:**
DynamoDB スロットリングは、アプリケーションがテーブルのプロビジョニングされた読み取り/書き込み容量を超えた場合に発生します。これは以下の場合に発生する可能性がある一般的な本番環境の問題です:
- トラフィックスパイクが予期せずプロビジョニングされた容量を超える
- テーブルの容量ユニットが不十分な状態で誤って設定されている

**障害の注入方法:**
`inject_dynamodb_throttling()` ヘルパー関数は以下によってこれをシミュレートします:
- メトリクステーブルを `PAY_PER_REQUEST`（無制限）から `PROVISIONED` 課金モードに変換
- 極めて低い容量制限を設定: **1 Read Capacity Unit** と **1 Write Capacity Unit**
- これはテーブルが毎秒約1回の読み取りと1回の書き込み操作しか処理できないことを意味します
- 通常のアプリケーション負荷はすぐにこれらの制限を超えます

**予想される影響:**
- アプリケーションログに `ProvisionedThroughputExceededException` エラー
- リクエストがスロットリングされリトライされるためレイテンシーが増加
- CloudWatch メトリクスにスロットリングされたリクエストが表示される

In [ ]:
# DynamoDB スロットリング障害注入を実行
success = inject_dynamodb_throttling(resources, AWS_REGION, AWS_PROFILE)

if success:
    print("✅ DynamoDBスロットリング障害が正常に注入されました")
    print("   → テーブルが1 RCU/1 WCUのPROVISIONEDモードに変換されました")
    print("   → 通常のアプリケーション負荷でスロットリングが発生するようになりました")
else:
    print("❌ DynamoDBスロットリング障害の注入に失敗しました")

### アプリケーションテーブルへの負荷をかける

それでは、エンドポイントに負荷テストを実行しましょう。30秒間、毎秒20の同時リクエストを送信します。この負荷は非常に大きくはありませんが、テーブル容量のプロビジョニングの誤設定により、アプリは[アプリケーションログ](https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logsV2:log-groups/log-group/$252Faws$252Fsre-workshop$252Fcrm-application)に `ProvisionedThroughputExceededException` エラーを表示し、CloudWatch メトリクスにスロットリングされたリクエストが表示されるはずです。負荷テスト中に Customers タブにアクセスしようとすると、データの読み込みに問題が発生します。

In [ ]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor

alb_dns = resources['public_alb_dns']
url = f"http://{alb_dns}:8080/api/customers"

def make_request(i):
    try:
        requests.get(url, timeout=5)
    except:
        pass

for second in range(1, 31):
    with ThreadPoolExecutor(max_workers=50) as executor:
        executor.map(make_request, range(50))

    if second % 10 == 0:
        print(f"進捗: {second}/30秒")

    time.sleep(1)

print("\n✓ 負荷テスト完了")

### 障害 2: IAM 権限の問題

**概要:**
IAM 権限の問題は、アプリケーションが AWS リソースにアクセスするために必要な権限を持っていない場合に発生します。これは最も一般的な本番環境の問題の1つで、多くの場合以下が原因です:
- テストせずに適用された過度に制限的なセキュリティポリシー
- 信頼ポリシーの変更によるロール引き受けの失敗
- セキュリティチームによる一括拒否ポリシーの適用

**障害の注入方法:**
ヘルパー関数 `inject_iam_permissions()` は以下によってこれをシミュレートします:
- アプリケーションサーバーが使用する EC2 インスタンス IAM ロールを特定
- 元の DynamoDB アクセスポリシーをバックアップ
- 主要な DynamoDB 操作に対して明示的な **Deny** ポリシーに置き換え
- 対象: `PutItem`、`GetItem`、`Query`、`Scan`、`UpdateItem`、`DeleteItem`
- Deny ポリシーは Allow ポリシーを上書きするため、これによりデータベースアクセスが即座にブロックされます

**予想される影響:**
- データベース操作に対するアプリケーションログの `AccessDenied` 例外
- DynamoDB アクセスを必要とする機能の完全な障害

In [ ]:
# IAM 権限障害注入を実行
success = inject_iam_permissions(resources, AWS_REGION, AWS_PROFILE)

if success:
    print("✅ IAM権限障害が正常に注入されました")
    print(f"   → EC2ロール '{resources.get('ec2_role_name', 'Unknown')}' にDenyポリシーが設定されました")
    print("   → すべてのDynamoDB操作がAccessDeniedを返すようになりました")
else:
    print("❌ IAM権限障害の注入に失敗しました")

アプリケーションを呼び出した際にどのようなレスポンスが得られるかテストしてみましょう。API が DynamoDB からデータを取得できないバックエンドの問題により、エラー 500 が表示されるはずです。
**注意**: IAM 権限が伝播するまで1分程度かかる場合があります。500 エラーが表示されない場合は、しばらく待ってから再試行してください。

In [ ]:
time.sleep(180)
alb_dns = resources['public_alb_dns']

url = f"http://{alb_dns}:8080/api/deals"

print(f"\nIAMエラーをトリガーするために5回のリクエストを生成中...")
print(f"ターゲット: {url}\n")

for i in range(10):
    try:
        response = requests.get(url, timeout=5)
        print(f"リクエスト {i+1} - ステータス: {response.status_code}")
    except Exception as e:
        print(f"リクエスト {i+1} - エラー: {str(e)}")

    time.sleep(1)  # 過負荷を避けるための小さな遅延

print("\n✓ 負荷完了 - ログが伝播するまで10秒待機中...")
time.sleep(10)

## まとめ

✅ 前提条件が確認され、インフラストラクチャがデプロイされました。CRM アプリケーションが稼働し、CloudWatch で監視されています。エージェントがトラブルシューティングするための実際の本番環境の問題をシミュレートする障害を注入しました。

次へ: Lab 2 - 診断エージェントの構築 (Lab-02-diagnostics-agent.ipynb)

In [ ]:
# ポート 80 と 8080 の両方で URL を試す
print(f"CRMアプリUIにアクセスするにはこちらをクリック: '{get_app_url()}'")